In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")

import json
import numpy as np
import pandas as pd
import torch

from constants import ( 
    MODEL_DIR,
    N_LEADS
)
from evaluate_script import read_data, predict
from resnet import ResNet1d

In [3]:
n_total = 10
# If a small array of traces IS already saved, read the np file
data_array = np.load('trace_file.npy')

# If a small array of traces IS NOT already saved, run the following line
# data_array, df, exam_ids = read_data(n_total=n_total, data_file='trace_file.npy')

In [4]:
data_array.shape[1:] == (4096, 12)

True

In [5]:
hidden_outputs = {}

def save_output(name):
    def hook(module, inp, out):
        hidden_outputs[name] = out.detach().cpu()
    return hook

In [6]:
config = f'{MODEL_DIR}/config.json'
with open(config, 'r') as f:
    config_dict = json.load(f)

model = ResNet1d(
    input_dim=(N_LEADS, config_dict['seq_length']),
    blocks_dim=list(zip(config_dict['net_filter_size'], 
                        config_dict['net_seq_lengh'])),
    n_classes=1,
    kernel_size=config_dict['kernel_size'],
    dropout_rate=config_dict['dropout_rate']
)

In [7]:
# Loading the model weights in the model object
state_dict = (torch.load(
    '../model/model.pth',
    weights_only=False,
    map_location=torch.device('cpu')
))

In [8]:
weights_metadata = []
for key, val in state_dict['model'].items():
    weights_metadata.append([key, val.shape])
weights_metadata = pd.DataFrame(weights_metadata, columns=['layer_name', 'layer_shape'])
weights_metadata

,layer_name,layer_shape
0,conv1.weight,"(64, 12, 17)"
1,bn1.weight,"(64,)"
2,bn1.bias,"(64,)"
3,bn1.running_mean,"(64,)"
4,bn1.running_var,"(64,)"
...,...,...
67,resblock1d_4.bn2.running_var,"(320,)"
68,resblock1d_4.bn2.num_batches_tracked,()
69,resblock1d_4.skip_connection.1.weight,"(320, 256, 1)"
70,lin.weight,"(1, 5120)"


In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Load the state dict and set the model to eval mode.
model.load_state_dict(state_dict['model'])
type(model)

resnet.ResNet1d

In [10]:
# Hook for initial layers
model.conv1.register_forward_hook(save_output("conv1"))
model.bn1.register_forward_hook(save_output("bn1"))

# Hooks for each ResBlock1d
for i, blk in enumerate(model.res_blocks):
    blk.conv1.register_forward_hook(save_output(f"block{i}_conv1"))
    blk.bn1.register_forward_hook(save_output(f"block{i}_bn1"))
    blk.conv2.register_forward_hook(save_output(f"block{i}_conv2"))
    blk.bn2.register_forward_hook(save_output(f"block{i}_bn2"))


In [11]:
out = model(torch.from_numpy(data_array).transpose(-1, -2))

In [12]:
layer0 = hidden_outputs["block0_conv1"]
print(layer0.shape)

torch.Size([10, 64, 4096])


In [13]:
hidden_outputs_metadata = []
for key, val in hidden_outputs.items():
    hidden_outputs_metadata.append([key, val.shape])
import pandas as pd
hidden_outputs_metadata = pd.DataFrame(hidden_outputs_metadata, columns=['layer_name', 'layer_shape'])
hidden_outputs_metadata

,layer_name,layer_shape
0,conv1,"(10, 64, 4096)"
1,bn1,"(10, 64, 4096)"
2,block0_conv1,"(10, 64, 4096)"
3,block0_bn1,"(10, 64, 4096)"
4,block0_conv2,"(10, 64, 4096)"
5,block0_bn2,"(10, 64, 4096)"
6,block1_conv1,"(10, 128, 4096)"
7,block1_bn1,"(10, 128, 4096)"
8,block1_conv2,"(10, 128, 1024)"
9,block1_bn2,"(10, 128, 1024)"
